## Importations

In [16]:
import numpy as np
import pandas as pd
import itertools as it
import scipy
from tqdm import tqdm
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer

# Global variables

In [17]:
PATH = "../data/ml-latest-small/"
TRAIN_SIZE = 0.7
MINIMUM_SEEN_MOVIES = 5

## Loading data

In [18]:
def load_data(path):
	movies = pd.read_csv(path+"movies.csv")
	ratings = pd.read_csv(path+"ratings.csv")
	return movies, ratings

In [4]:
movies, ratings = load_data(PATH)

# Preprocessing data

In [23]:
## Delete films seen only by a certain number of users
def delete_solo_films(movies, ratings, threshold):
	list_key_values = np.array(list(Counter(ratings['movieId']).items()))
	_, values = list_key_values[:,0], list_key_values[:,1]
	index = np.where(values > threshold)
	return movies[movies["movieId"] != str(index)], ratings[ratings["movieId"] != str(index)]

## Building ratings and movies dataframe which both contains the same movieId
def clear_dataset(movies, ratings):
	movies, ratings = delete_solo_films(movies, ratings, MINIMUM_SEEN_MOVIES)

	list_movieId = set(movies["movieId"]).intersection(set(ratings["movieId"]))

	l = []
	for i in range(len(movies['movieId'])):
		if movies['movieId'][i] in list_movieId:
			l.append(i)
	movies = movies.iloc[l,:]

	a = sorted(list(list_movieId))
	b = range(len(a))
	d = dict(zip(a,b))
	movies = movies.replace({'movieId' : d})

	a = sorted(list(list_movieId))
	b = range(len(a))
	d = dict(zip(a,b))
	ratings = ratings.replace({'movieId' : d})

	return movies, ratings	
	
## Building one hot encoded genres in movies dataframe
def one_hot_encode_genres(movies):
	tmp = []
	for elt in movies["genres"]:
		tmp.append(elt.split("|"))
	movies["genres"] = tmp
	mlb = MultiLabelBinarizer(sparse_output=True)
	movies = movies.join(
				pd.DataFrame.sparse.from_spmatrix(
												mlb.fit_transform(movies.pop('genres')),
												index=movies.index,
												columns=mlb.classes_))
	return movies

## Cleaning ratings datagrame
def preprocess_ratings(ratings):
	ratings = ratings.drop(columns=["timestamp"])
	ratings['userId'] = ratings['userId'].to_numpy() - 1 # car pas de user 0
	return ratings

## Split for computing metrics on test later
def split_set(userId, train_size, ratings):
	rating_user = ratings[ratings["userId"] == userId]
	train_rating_user, test_rating_user = rating_user.to_numpy()[:int(train_size*len(rating_user))], rating_user.to_numpy()[int(train_size*len(rating_user)):]
	return train_rating_user, test_rating_user

## Get informations on users watched/unwatched movies...
def get_infos_user(userId):
	watched_user = set(ratings[ratings["userId"] == userId]["movieId"])
	watched_all = set(ratings['movieId'])
	unwatched_user = list(watched_all.difference(watched_user))
	return watched_user, watched_all, unwatched_user

In [24]:
movies, ratings = clear_dataset(movies, ratings)
movies = one_hot_encode_genres(movies)
ratings = preprocess_ratings(ratings)

KeyError: 816

# Building matrix

In [14]:
# Building a sparse matrix which contains the triple (u_k, m_i, r_ki)
# def build_sparse_matrix_triples(ratings):
# 	ratings_sparse = scipy.sparse.csr_matrix(ratings.values)
# 	return ratings_sparse

## Building a matrix M = (n_movies, n_movies) which contains the number of users who'se seen m_i and m_j
def build_M_matrix(ratings, train_size):
	data_dict = dict()
	test_rating_user_list = []
	for userId in tqdm(set(ratings["userId"])):
		train_rating_user, test_rating_user = split_set(userId, train_size, ratings)
		test_rating_user_list.append(test_rating_user)
		iterator = it.combinations(train_rating_user[:,1], 2)
		for x, y in iterator:
			data_dict[(x,y)] = data_dict.get((x,y), 0) + 1
			data_dict[(y,x)] = data_dict.get((y,x), 0) + 1
		iterator = it.combinations(test_rating_user[:,1], 2)
		for x, y in iterator:
			## We need to ignore the test movies
			data_dict[(x,y)] = 0
			data_dict[(y,x)] = 0
			# data_dict[(x,y)] = 1
			# data_dict[(y,x)] = 1
	keys = np.array(list(data_dict.keys())).astype(int)
	values = np.array(list(data_dict.values())).astype(int)
	M_coo = scipy.sparse.coo_matrix((values, (keys[:,0], keys[:,1])))
	M_csr = M_coo.tocsr()
	print(M_csr)
	M_norm = M_csr / M_csr.sum(axis=0)
	# M_norm = np.divide(M_csr, M_csr.sum(axis=0), where=M_csr.sum(axis=0)!=0)
	print(M_norm)
	return M_norm, test_rating_user_list

## Computing probabilites of genres P_ig
def build_P_ig(movies):
	sum_ = movies[[i for i in movies.columns if i != "movieId" and i != "title"]].to_numpy().sum(axis=0).astype(int)
	P_ig = sum_ / sum(sum_)
	return P_ig.reshape(-1, 1)

## Initialisation of R_uk before iterative algorithm
def init_R_uk(movies):
	n_genres = len(movies.columns) - 2
	n_movies = len(movies)
	r = 1/(n_movies*n_genres)
	R_uk = np.full((n_movies, n_genres), r)
	return R_uk

## Computing F_ig for each user
def build_F_ig(R_uk, P_ig):
	F_ig = np.sum(R_uk, axis=1).reshape(-1,1) @ P_ig.reshape(1,-1)
	return F_ig

## Matrix user X movie
def build_ratings_matrix(ratings):
	values = ratings["rating"]
	rows = ratings["userId"]
	cols = ratings["movieId"]
	M_coo = scipy.sparse.coo_matrix((values, (rows, cols)))
	M_csr = M_coo.tocsr()
	return M_csr

## Build I_uk for each user
def build_I_uk(tmp_M, id_user, P_ig):
# 	print(tmp_M[id_user,:].T.shape)
	I_uk = tmp_M[id_user,:].T @ P_ig.reshape(1,-1)
	I_uk = I_uk / I_uk.sum(axis=0).T
	return I_uk

## Init the matrix needed before running the iterative algorithm
def init(movies, ratings, train_size):
	print("Init R_uk...")
	R_uk = init_R_uk(movies)
	# print(R_uk.shape)
	print("Building P_ig...")
	tmp_M = build_ratings_matrix(ratings)
	P_ig = build_P_ig(movies)
	# print(P_ig.shape)
	print("Building M_csr...")
	M_csr, test_rating_user_list = build_M_matrix(ratings, train_size)
	# print(M_csr.shape)
	return R_uk, tmp_M, P_ig, M_csr, np.array(test_rating_user_list, dtype=object)

In [15]:
R_uk, tmp_M, P_ig, M_csr, test_rating_user_list = init(movies, ratings, TRAIN_SIZE)

Init R_uk...
Building P_ig...
Building M_csr...


100%|██████████| 610/610 [00:18<00:00, 32.76it/s]


  (0, 1)	68
  (0, 2)	32
  (0, 3)	2
  (0, 4)	32
  (0, 5)	58
  (0, 6)	32
  (0, 7)	5
  (0, 8)	12
  (0, 9)	69
  (0, 10)	42
  (0, 11)	15
  (0, 12)	6
  (0, 13)	12
  (0, 14)	8
  (0, 15)	41
  (0, 16)	44
  (0, 17)	12
  (0, 18)	55
  (0, 19)	8
  (0, 20)	47
  (0, 21)	22
  (0, 22)	11
  (0, 23)	18
  (0, 24)	42
  (0, 25)	10
  :	:
  (9723, 7870)	1
  (9723, 7876)	1
  (9723, 7937)	1
  (9723, 7961)	1
  (9723, 8005)	1
  (9723, 8045)	1
  (9723, 8133)	1
  (9723, 8278)	1
  (9723, 8285)	1
  (9723, 8287)	1
  (9723, 8295)	1
  (9723, 8354)	1
  (9723, 8448)	1
  (9723, 8449)	1
  (9723, 8457)	1
  (9723, 8532)	1
  (9723, 8551)	1
  (9723, 8618)	1
  (9723, 8663)	1
  (9723, 8673)	1
  (9723, 8861)	1
  (9723, 9374)	1
  (9723, 9603)	1
  (9723, 9665)	1
  (9723, 9691)	1
[[0.         0.0021645  0.00208578 ... 0.         0.         0.        ]
 [0.00153246 0.         0.00169469 ... 0.         0.         0.        ]
 [0.00072116 0.0008276  0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.        

## Run the algorithm

In [ ]:
## Compute TR_ki for a specific user
def compute_TR_ki(id_user, R_uk, tmp_M, P_ig, M_csr, d, alpha, iter_max):
	I_uk = build_I_uk(tmp_M, id_user, P_ig)
	for _ in range(iter_max):
		F_ig = build_F_ig(R_uk, P_ig)
		R_uk = d * alpha * M_csr @ R_uk + d * (1-alpha) * M_csr @ F_ig + (1-d) * I_uk

		# This part is useful if you want to normalize + break if converge
		# R_uk = (R_uk / R_uk.sum(axis=1)).T # Normalization isn't working
		#     print(np.abs(np.sum(R_uk - R_uk_old)))
		#     if np.abs(np.sum(R_uk - R_uk_old)) < eps :
		#         print(i)
		#         break
		# R_uk_old = R_uk.copy()
	
	TR_ki = np.array(R_uk @ P_ig) # It returns a np.mat object which can't be reduced to dimension 1
	return TR_ki.reshape(-1)

## Compute TR_ki for all users
def iterative_TR_ki(n_user, R_uk, tmp_M, P_ig, M_csr, d=0.15, alpha=0.1, iter_max=5):
	print("Computing TR_ki for all users...")
	TR_ki_all_user = []
	for id_user in tqdm(range(n_user)):
		TR_ki_all_user.append(compute_TR_ki(id_user, R_uk, tmp_M, P_ig, M_csr, d, alpha, iter_max))
	return np.array(TR_ki_all_user)

In [ ]:
# n_user = len(np.unique(ratings["userId"]))
n_user = 2

TR_ki_all_user = iterative_TR_ki(n_user, R_uk, tmp_M, P_ig, M_csr, d=0.15, alpha=0.1, iter_max=5)

Computing TR_ki for all users...


100%|██████████| 2/2 [00:49<00:00, 24.84s/it]


## Running some test for a test user

In [ ]:
## Returns the recommandation for the users
def sort_by_best_movie(TR_ki_all_user):
	sorted_movies_all_user = np.zeros_like(TR_ki_all_user)
	for i in range(len(TR_ki_all_user)):
		sorted_movies_all_user[i,:] = np.argsort(TR_ki_all_user[i,:])[::-1]
	return sorted_movies_all_user

In [ ]:
test_user_id = 1

print("TR_ki_all_user shape:", TR_ki_all_user.shape)
print("test_rating_user_list shape:", test_rating_user_list.shape)
print("TR_ki for test user :\n", TR_ki_all_user[test_user_id, :10])

TR_ki_all_user shape: (2, 9724)
test_rating_user_list shape: (610,)
TR_ki for test user :
 [nan nan nan nan nan nan nan nan nan nan]


In [ ]:
sorted_movies_all_user = sort_by_best_movie(TR_ki_all_user)
print("sorted_movies_all_user shape:", sorted_movies_all_user.shape)
print("Sorted best movies recommandation for test user :\n", sorted_movies_all_user[test_user_id,:10])

sorted_movies_all_user shape: (2, 9724)
Sorted best movies recommandation for test user :
 [9723. 3246. 3244. 3243. 3242. 3241. 3240. 3239. 3238. 3237.]


## Running some test for the DOA metrics before computing it on whole user list

In [ ]:
## Computes DOA score for a specific user
def compute_doa_score(TR_ki, test_rating_user, unwatched_user):
	score = 0
	for m_i in test_rating_user:
		for m_j in unwatched_user:
			if TR_ki[int(m_i)] > TR_ki[int(m_j)]:
				score += 1
	return score / (len(test_rating_user) * len(unwatched_user))

## Computes DOA for the whole user list
def compute_all_doa(TR_ki_all_user, test_rating_user_list):
	score = 0
	n_user = TR_ki_all_user.shape[0]
	for user_id in range(n_user):
		tmp = test_rating_user_list[user_id]
		test_films = tmp[:,1]
		TR_ki = TR_ki_all_user[user_id]
		_, _, unwatched_films = get_infos_user(user_id)
		score += compute_doa_score(TR_ki, test_films, unwatched_films)
	return score / n_user

def test_doa_user(n_user, n_film_test):
	for test_user_id in range(n_user):
		print("---------- Testing user number", test_user_id, "----------")
		test = test_rating_user_list[test_user_id]
		test_films = test[:,1]
		TR_ki = TR_ki_all_user[test_user_id]
		_, _, unwatched_films = get_infos_user(test_user_id)
		print("Some movies from test set:")
		print(test_films[:n_film_test])
		print("Some movies from unwatched set:")
		print(unwatched_films[:n_film_test])
		print("Some score for test set (movies watched by user):")
		for i in test_films[:n_film_test]:
			print(TR_ki[int(i)])
		print("Some score for unwatched movies:")
		for i in unwatched_films[:n_film_test]:
			print(TR_ki[int(i)])
		print("Total DOA score for this user :", compute_doa_score(TR_ki, test_films, unwatched_films))

In [ ]:
n_film_test = 3

test_doa_user(n_user, n_film_test)

---------- Testing user number 0 ----------
Some movies from test set:
[1882. 1903. 1904.]
Some movies from unwatched set:
[1, 3, 4]
Some score for test set (movies watched by user):
nan
nan
nan
Some score for unwatched movies:
nan
nan
nan
Total DOA score for this user : 0.0
---------- Testing user number 1 ----------
Some movies from test set:
[7758. 8045. 8287.]
Some movies from unwatched set:
[0, 1, 2]
Some score for test set (movies watched by user):
nan
nan
nan
Some score for unwatched movies:
nan
nan
nan
Total DOA score for this user : 0.0


## Computing final DOA metric for the whole dataset

In [ ]:
print("DOA for all users :", compute_all_doa(TR_ki_all_user, test_rating_user_list))

DOA for all users : 0.0


## Dimension & explication

In [ ]:
# uk -> user k 
# ig -> movies i, genre g 
# R_uk -> movie,genre pour user uk
# P_ig -> désigne la probabilité avec laquelle l'item i appartient au genre g
# M -> matrice correlation
# F -> 
# I_uk -> 
# r_ki -> 

#R -> n_user X n_movies X n_genres
#I -> n_user X n_movies X n_genres

#M -> n_movies X n_movies
#F_ig -> n_movies X n_genres
#I_uk -> n_movies X n_genres

# Split train test sur les users????????
# Entrainement sur le train 
# Test a la fin comparaison NDCG entre tri du ranking et notes de l'utilisateur 

# Split train test sur les ratings par users (on ignore les ratings test pour le train)
# Entrainement sur le train (il reste le meme, on a juste caché des notes)
# Test a la fin comparaison NDCG entre tri du ranking (donné par l'algo grace au train) et notes de l'utilisateur (qu'on a garder en test)
# Test avec le DOA entre les films (qu'on a cache) et les films unwatched ?

# On prend pour un utilisateur sa liste de film + ses notes
# On montre cette liste a des nouveaux utilisateurs puis on leur demande parmis les films que notre algo sort quel est celui qui devrait etre recommandé
# On compare les resultats avec notre ranking reel apres train
# Test de student 